In [1]:
# goto /home/macul/libraries/MXNet2Caffe
# run python -m json2prototxt --mx-json /media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-symbol.json --cf-prototxt /media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe-converted.prototxt
# change bottom: "_mulscalar0" to bottom: "data"
# add below layer after bn1_scale
'''
layer {
  bottom: "bn1"
  top: "flatten"
  name: "fc1_flatten"
  type: "Flatten"
  flatten_param { axis: 1}
}
'''
# save modified file as model-caffe.prototxt
# run python -m mxnet2caffe --mx-model /media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model  --cf-prototxt /media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe.prototxt --cf-model /media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe-converted.caffemodel

'\nlayer {\n  bottom: "bn1"\n  top: "flatten"\n  name: "fc1_flatten"\n  type: "Flatten"\n  flatten_param { axis: 1}\n}\n'

In [2]:
import sys
sys.path.append('/home/macul/libraries/MXNet2Caffe')
sys.path.append('/home/macul/insightface/src/')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxiter')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxconv')
import mxnet as mx
import caffe
from collections import namedtuple
import numpy as np
import cv2

In [3]:
mxModelPath = '/media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model'
mxModelEpoch = 0

sym_tmp, arg_params, aux_params = mx.model.load_checkpoint(mxModelPath, mxModelEpoch)

In [4]:
all_layers = sym_tmp.get_internals()
output_list = all_layers.list_outputs()
sym3 = all_layers['fc1_output']
model = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
model.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])
model.set_params(arg_params, aux_params, allow_missing=False, allow_extra=True)

In [5]:
output_list

['data',
 'id_output',
 '_minusscalar0_output',
 '_mulscalar0_output',
 'conv0_weight',
 'conv0_output',
 'bn0_gamma',
 'bn0_beta',
 'bn0_moving_mean',
 'bn0_moving_var',
 'bn0_output',
 'relu0_gamma',
 'relu0_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_relu1_gamma',
 'stage1_unit1_relu1_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn3_gamma',
 'stage1_unit1_bn3_beta',
 'stage1_unit1_bn3_moving_mean',
 'stage1_unit1_bn3_moving_var',
 'stage1_unit1_bn3_output',
 'stage1_unit1_conv1sc_weight',
 'stage1_unit1_conv1sc_output',
 'stage1_unit1_sc_gamma',
 'stage1_unit1_sc_beta',
 'stage1_unit1_sc_moving_mean',
 'stage1_unit1_sc

In [5]:
img = mx.image.imread('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0000.png')
img = img.astype('float32')
#if img_normalization:
#    img = img - self.mean_value
#    img = img * self.scale_value
img = img.transpose((2,0,1))
img = img.expand_dims(axis=0)   
Batch = namedtuple('Batch',['data'])
model.forward(Batch([img]))

In [6]:
outs=model.get_outputs()

In [7]:
out_mx=outs[0].asnumpy()[0]

In [8]:
print(arg_params['pre_fc1_bias'].shape)
print(arg_params['pre_fc1_weight'].shape)

(512L,)
(512L, 25088L)


In [9]:
cfModelProto='/media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe.prototxt'
cfModelWeight='/media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe-converted.caffemodel'
Net = caffe.Net(cfModelProto, cfModelWeight, caffe.TEST)
Net.blobs['data'].reshape(1,3,112,112)

In [10]:
def getOutput(img_path, output_name):
    mean_array = np.ones([3,112, 112])*127.5
    scale_value = 0.0078125
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    img = np.rollaxis(img,2)
    img = img - mean_array
    img = img * scale_value
    img = img[np.newaxis,...] 
    
    Net.blobs['data'].data[...] = img
    out = Net.forward()
    out = out[output_name][0]
    return out

In [12]:
Net.params['pre_fc1'][1].data[...] = arg_params['pre_fc1_bias'].asnumpy()

In [13]:
Net.params['pre_fc1'][0].data[...] = arg_params['pre_fc1_weight'].asnumpy()

In [14]:
out_caffe = getOutput('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0000.png', 'fc1')

In [15]:
out_caffe

array([-2.09693149e-01,  8.06518555e-01, -1.62811488e-01,  3.02189380e-01,
       -8.14458668e-01, -2.00440884e+00,  5.09258747e-01,  2.39765406e-01,
        9.20235276e-01,  8.25070590e-02,  4.78360474e-01,  1.01424120e-01,
        2.10276985e+00,  4.21942800e-01,  1.99278820e+00, -5.66515923e-01,
       -1.12407422e+00, -1.54677093e+00,  3.71641189e-01,  1.67366791e+00,
        1.39175367e+00,  4.55739319e-01,  5.52479148e-01,  1.69869316e+00,
        1.62767100e+00, -1.11143804e+00,  7.23164558e-01,  5.47010720e-01,
       -1.18753541e+00,  6.22065067e-01, -1.02715576e+00, -5.15784860e-01,
       -3.86796713e-01,  8.72745097e-01,  3.75263780e-01, -1.75003216e-01,
       -1.07182771e-01,  4.43520814e-01,  5.87100208e-01, -6.75549448e-01,
        2.93192416e-01,  1.16836119e+00,  4.77570176e-01,  3.60459954e-01,
       -6.64982080e-01, -7.13824153e-01,  1.74423099e-01,  7.60683358e-01,
        3.46772701e-01,  1.68500686e+00,  1.20626318e+00,  1.93103611e+00,
       -7.82145977e-01, -

In [16]:
np.linalg.norm(out_caffe)

21.982462

In [17]:
np.dot(out_caffe/np.linalg.norm(out_caffe),out_mx/np.linalg.norm(out_mx))

1.0

In [18]:
Net.save('/media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe.caffemodel')

In [19]:
cfModelProto='/media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe.prototxt'
cfModelWeight='/media/macul/black/mxnet_training/r50/insightface-r50-am-lfw/model-caffe.caffemodel'
Net = caffe.Net(cfModelProto, cfModelWeight, caffe.TEST)
Net.blobs['data'].reshape(1,3,112,112)

In [20]:
out_caffe_new = getOutput('/media/macul/black/face_database_raw_data/mscelb_from_insightface/Data/0000000/0000.png', 'fc1')

In [21]:
np.dot(out_caffe_new/np.linalg.norm(out_caffe_new),out_mx/np.linalg.norm(out_mx))

1.0

In [ ]:
# model-caffe.prototxt and model-caffe.caffemodel are converted caffe model for use
# note that the input channel should be RGB, image should be normalized, and the output embedding should be normalized by its norm